In [1]:
import pandas as pd
from pathlib import Path
import re

In [10]:
# --- 1. Где лежат ваши файлы -----------------------------------------------
DATA_DIR = Path("../data_in_world_frame/data_in_world_frame/datasets")      # при необходимости поменяйте путь
PATTERN   = "dataset_raw*.csv"       

def natural_key(p):                     # сортировка raw1, raw2, …
    parts = re.split(r"(\d+)", p.stem)
    return [int(s) if s.isdigit() else s for s in parts]

dfs_circle = []
for f in sorted(DATA_DIR.glob(PATTERN), key=natural_key):
    df = pd.read_csv(
        f,
        engine="python"     # терпимее к кривым строкам
    )
    dfs_circle.append(df[1:])

       t      ax      ay   yawRate  steer  t_sec         yaw    yaw_rad  \
0   1040 -0.0134  0.1231    0.5488   2700  1.040    0.000000   0.000000   
1   1202  0.1118  0.2270    0.2134   2820  1.202    0.034571   0.000603   
2   1255  0.0631  0.3109    1.7378   2970  1.255    0.126674   0.002211   
3   1309  0.1261  0.3698    0.7317   2948  1.309    0.166186   0.002900   
4   1363  0.3102  0.5118   -0.0610   2168  1.363    0.162892   0.002843   
..   ...     ...     ...       ...    ...    ...         ...        ...   
71  4980  2.3273  2.4586 -297.4695   1966  4.980 -605.519244 -10.568304   
72  5035  2.2454  2.3054 -300.8842   1968  5.035 -622.067875 -10.857133   
73  5089  1.4640  2.4443 -295.3964   1968  5.089 -638.019281 -11.135537   
74  5143  1.2038  2.6919 -291.0976   1968  5.143 -653.738551 -11.409890   
75  5196  1.0047  2.7463 -297.5610   1968  5.196 -669.509284 -11.685141   

    ax_world  ay_world  angular_accel  
0  -0.013400  0.123100       0.000000  
1   0.111663  0.227

In [11]:
dfs_circle[0]

,t,ax,ay,yawRate,steer,t_sec,yaw,yaw_rad,ax_world,ay_world,angular_accel
1,1202,0.1118,0.2270,0.2134,2820,1.202,0.034571,0.000603,0.111663,0.227067,-2.070370
2,1255,0.0631,0.3109,1.7378,2970,1.255,0.126674,0.002211,0.062412,0.311039,28.762264
3,1309,0.1261,0.3698,0.7317,2948,1.309,0.166186,0.002900,0.125027,0.370164,-18.631481
4,1363,0.3102,0.5118,-0.0610,2168,1.363,0.162892,0.002843,0.308744,0.512680,-14.679630
5,1418,0.2596,0.5095,1.2195,1968,1.418,0.229964,0.004014,0.257553,0.510538,23.281818
...,...,...,...,...,...,...,...,...,...,...,...
71,4980,2.3273,2.4586,-297.4695,1966,4.980,-605.519244,-10.568304,-3.201977,1.099264,-41.992453
72,5035,2.2454,2.3054,-300.8842,1968,5.035,-622.067875,-10.857133,-2.593207,1.905772,-62.085455
73,5089,1.4640,2.4443,-295.3964,1968,5.089,-638.019281,-11.135537,-2.216160,1.790679,101.625926
74,5143,1.2038,2.6919,-291.0976,1968,5.143,-653.738551,-11.409890,-1.979537,2.185610,79.607407


In [34]:
import pandas as pd
from pathlib import Path
import re

# --- 1. Где лежат ваши файлы -----------------------------------------------
DATA_DIR = Path("../data_in_world_frame/data_in_world_frame/datasets")  # при необходимости поменяйте путь
PATTERN = "dataset_raw*.csv"       

def natural_key(p):  # сортировка raw1, raw2, …
    parts = re.split(r"(\d+)", p.stem)
    return [int(s) if s.isdigit() else s for s in parts]

dfs_circle = []

for f in sorted(DATA_DIR.glob(PATTERN), key=natural_key):
    df = pd.read_csv(f, engine="python")
    df = df[1:].copy()  # пропускаем первую строку (возможно, повреждена)

    # --- 2. Добавим столбец "is_drift" ---------------------------------------
    YAW_THR   = 150     # |°/с|
    AY_THR    =  2.5    # |g|   (~25 м/с²)
    ANGACC_THR= 100     # |°/с²| -- для отлова самого начала
    
    df["is_drift"] = (
            (df["yawRate"].abs()    > YAW_THR) |
            (df["ay_world"].abs()   > AY_THR ) &
            (df["angular_accel"].abs() > ANGACC_THR)
    ).astype(int)

    dfs_circle.append(df)

In [36]:
dfs_circle[1]

,t,ax,ay,yawRate,steer,t_sec,yaw,yaw_rad,ax_world,ay_world,angular_accel,is_drift
1,1092,-0.0446,0.1773,-0.6402,1982,1.092,-0.034571,-0.000603,-0.044493,0.177327,-23.148148,0
2,1145,-0.0544,0.2567,-10.1220,1968,1.145,-0.571037,-0.009966,-0.051839,0.257229,-178.901887,0
3,1200,-0.0598,0.7374,-13.7805,1968,1.200,-1.328964,-0.023195,-0.042682,0.738589,-66.518182,0
4,1254,-0.2504,1.0980,-42.7439,1968,1.254,-3.637135,-0.063480,-0.180241,1.111673,-536.359259,0
5,1308,-0.1644,1.9467,-66.7073,1968,1.308,-7.239329,-0.126350,0.082222,1.951898,-443.766667,0
...,...,...,...,...,...,...,...,...,...,...,...,...
57,4115,-0.0522,3.9117,-275.4878,1968,4.115,-709.630781,-12.385394,-0.755417,3.838420,182.362963,1
58,4169,0.2695,4.5893,-269.0549,1968,4.169,-724.159745,-12.638972,0.601686,4.557661,119.127778,1
59,4222,0.0509,4.6181,-265.8232,1968,4.222,-738.248375,-12.884865,1.494438,4.369908,60.975472,1
60,4277,0.3175,4.4903,-260.8842,1968,4.277,-752.597006,-13.135296,2.686532,3.611945,89.800000,1


In [32]:
# --- 1. Где лежат ваши файлы -----------------------------------------------
DATA_DIR = Path("../data_in_world_frame/data_in_world_frame/datasets_slide")  # при необходимости поменяйте путь
PATTERN = "dataset_raw*.csv"       

def natural_key(p):  # сортировка raw1, raw2, …
    parts = re.split(r"(\d+)", p.stem)
    return [int(s) if s.isdigit() else s for s in parts]

dfs_slide = []

for f in sorted(DATA_DIR.glob(PATTERN), key=natural_key):
    df = pd.read_csv(f, engine="python")
    df = df[1:].copy()  # пропускаем первую строку (возможно, повреждена)

    # --- 2. Добавим столбец "is_drift" ---------------------------------------
    YAW_THR   = 150     # |°/с|
    AY_THR    =  2.5    # |g|   (~25 м/с²)
    ANGACC_THR= 100     # |°/с²| -- для отлова самого начала
    
    df["is_drift"] = (
            (df["yawRate"].abs()    > YAW_THR) |
            (df["ay_world"].abs()   > AY_THR ) &
            (df["angular_accel"].abs() > ANGACC_THR)
    ).astype(int)

    dfs_slide.append(df)

In [33]:
dfs_slide[0]

,t,ax,ay,yawRate,steer,t_sec,yaw,yaw_rad,ax_world,ay_world,angular_accel,is_drift
1,1101,-0.0027,0.2319,0.4878,3394,1.101,0.025853,0.000451,-0.002805,0.231899,0.575472,0
2,1156,-0.0510,0.2602,3.0793,3742,1.156,0.195215,0.003407,-0.051886,0.260025,47.118182,0
3,1210,-0.1069,0.2608,9.5732,3906,1.210,0.712168,0.012430,-0.110133,0.259451,120.257407,0
4,1263,-0.1343,0.3202,11.4024,3996,1.263,1.316495,0.022977,-0.141621,0.317030,34.513208,0
5,1317,-0.1735,0.3644,11.1585,4004,1.317,1.919054,0.033494,-0.185606,0.358386,-4.516667,0
...,...,...,...,...,...,...,...,...,...,...,...,...
72,4934,-1.7186,0.4560,9.7866,3164,4.934,255.778200,4.464172,0.864244,1.553900,285.681481,0
73,4988,-1.3710,0.6770,-4.8780,3004,4.988,255.514788,4.459574,0.998408,1.158081,-271.566667,0
74,5043,-1.1658,0.8753,-4.9085,2980,5.043,255.244820,4.454863,1.143352,0.904426,-0.554545,0
75,5097,-1.1673,0.9047,-0.9756,2990,5.097,255.192138,4.453943,1.172990,0.897310,72.831481,0


In [30]:
# --- 1. Где лежат ваши файлы -----------------------------------------------
DATA_DIR = Path("../data_in_world_frame/data_in_world_frame/datasets_straight")  # при необходимости поменяйте путь
PATTERN = "dataset_raw*.csv"       

def natural_key(p):  # сортировка raw1, raw2, …
    parts = re.split(r"(\d+)", p.stem)
    return [int(s) if s.isdigit() else s for s in parts]

dfs_straight = []

for f in sorted(DATA_DIR.glob(PATTERN), key=natural_key):
    df = pd.read_csv(f, engine="python")
    df = df[1:].copy()  # пропускаем первую строку (возможно, повреждена)

    # --- 2. Добавим столбец "is_drift" ---------------------------------------
    YAW_THR   = 150     # |°/с|
    AY_THR    =  2.5    # |g|   (~25 м/с²)
    ANGACC_THR= 100     # |°/с²| -- для отлова самого начала
    
    df["is_drift"] = (
            (df["yawRate"].abs()    > YAW_THR) |
            (df["ay_world"].abs()   > AY_THR ) &
            (df["angular_accel"].abs() > ANGACC_THR)
    ).astype(int)

    dfs_straight.append(df)

In [37]:
dfs_straight[1]

,t,ax,ay,yawRate,steer,t_sec,yaw,yaw_rad,ax_world,ay_world,angular_accel,is_drift
1,1094,-0.0653,0.2628,0.5793,2978,1.094,0.031282,0.000546,-0.065443,0.262764,1.694444,0
2,1147,-0.0848,0.3668,0.4573,2980,1.147,0.055519,0.000969,-0.085155,0.366718,-2.301887,0
3,1202,-0.1339,0.4342,0.7012,2992,1.202,0.094085,0.001642,-0.134613,0.433980,4.434545,0
4,1256,-0.1641,0.4944,0.4878,2990,1.256,0.120426,0.002102,-0.165139,0.494054,-3.951852,0
5,1310,-0.1878,0.5406,0.5793,2978,1.310,0.151708,0.002648,-0.189231,0.540101,1.694444,0
...,...,...,...,...,...,...,...,...,...,...,...,...
80,5359,-5.4715,0.2636,186.9207,3004,5.359,107.617985,1.878288,1.404818,-5.294647,121.385185,1
81,5413,-5.0744,0.0605,193.3232,3000,5.413,118.057438,2.060491,2.333387,-4.506495,118.564815,1
82,5467,-5.6478,-0.3179,199.3902,2984,5.467,128.824508,2.248412,3.788482,-4.200727,112.351852,1
83,5521,-5.2202,-0.2768,216.4329,2982,5.521,140.511885,2.452395,4.204745,-3.105998,315.605556,1


In [40]:
n = 1
for df_i in dfs_circle:
    df_i.to_csv(f'../full_dataset/circle/dataset_raw{n}.csv', index=False)
    n += 1

In [41]:
n = 1
for df_i in dfs_slide:
    df_i.to_csv(f'../full_dataset/slide/dataset_raw{n}.csv', index=False)
    n += 1

In [42]:
n = 1
for df_i in dfs_straight:
    df_i.to_csv(f'../full_dataset/straight/dataset_raw{n}.csv', index=False)
    n += 1

In [44]:
dfs_circle[0]

,t,ax,ay,yawRate,steer,t_sec,yaw,yaw_rad,ax_world,ay_world,angular_accel,is_drift
1,1202,0.1118,0.2270,0.2134,2820,1.202,0.034571,0.000603,0.111663,0.227067,-2.070370,0
2,1255,0.0631,0.3109,1.7378,2970,1.255,0.126674,0.002211,0.062412,0.311039,28.762264,0
3,1309,0.1261,0.3698,0.7317,2948,1.309,0.166186,0.002900,0.125027,0.370164,-18.631481,0
4,1363,0.3102,0.5118,-0.0610,2168,1.363,0.162892,0.002843,0.308744,0.512680,-14.679630,0
5,1418,0.2596,0.5095,1.2195,1968,1.418,0.229964,0.004014,0.257553,0.510538,23.281818,0
...,...,...,...,...,...,...,...,...,...,...,...,...
71,4980,2.3273,2.4586,-297.4695,1966,4.980,-605.519244,-10.568304,-3.201977,1.099264,-41.992453,1
72,5035,2.2454,2.3054,-300.8842,1968,5.035,-622.067875,-10.857133,-2.593207,1.905772,-62.085455,1
73,5089,1.4640,2.4443,-295.3964,1968,5.089,-638.019281,-11.135537,-2.216160,1.790679,101.625926,1
74,5143,1.2038,2.6919,-291.0976,1968,5.143,-653.738551,-11.409890,-1.979537,2.185610,79.607407,1


In [46]:
dfs_circle[2]

,t,ax,ay,yawRate,steer,t_sec,yaw,yaw_rad,ax_world,ay_world,angular_accel,is_drift
1,1097,-0.0465,0.2744,0.8232,3822,1.097,0.043630,0.000761,-0.046709,0.274365,4.601887,0
2,1152,-0.3868,0.4738,6.8902,3340,1.152,0.422591,0.007376,-0.390284,0.470934,110.309091,0
3,1206,-0.7940,0.6994,22.7439,3000,1.206,1.650761,0.028811,-0.813818,0.676237,293.587037,0
4,1259,-0.8708,1.3285,35.9756,2984,1.259,3.557468,0.062090,-0.951555,1.271907,249.654717,0
5,1313,-1.4154,2.0483,23.6280,2982,1.313,4.833380,0.084358,-1.582953,1.921757,-228.659259,0
...,...,...,...,...,...,...,...,...,...,...,...,...
66,4606,-5.1073,1.5653,297.4390,4002,4.606,844.768419,14.743990,1.626656,-5.088091,-6.775926,1
67,4660,-5.4898,0.9338,311.1281,4002,4.660,861.569336,15.037222,3.720075,-4.143782,253.501852,1
68,4714,-4.9320,1.2656,313.7805,4002,4.714,878.513483,15.332953,4.125678,-2.984150,49.118519,1
69,4767,-4.7098,1.5924,319.6951,3670,4.767,895.457323,15.628679,4.568884,-1.960421,111.596226,1
